# NLDN CSV to Parquet

## This notebook converts monthly CSV-formatted files of real-time lightning flashes/strikes as reported by the [National Lightning Detection Network](https://www.vaisala.com/en/products/national-lightning-detection-network-nldn) (NLDN, originally developed in our department in the 1980s!) into Parquet (a binary format ideally suited for tabular datasets, especially when hosted on cloud platforms) and then visualizes data from the Parquet file.
### We will also compare the performance (i.e., time to execute) of reading in these files, as well as code cells in general, via the use of one of Jupyterlab's *cell magic* directives.

## Imports

In [ ]:
import pandas as pd
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import geopandas as gpd
import polars as pl

Specify the first day of a month.

In [ ]:
current = datetime(2025,8,1)
month = current.strftime("%Y%m")
month

Define an object that points to the August 2025 NLDN data file in CSV format.

In [ ]:
NLDN_csv = f'/spare11/atm533/data/NLDN_{month}.txt'

Take a peek at the first five lines of the file, using the **!** directive to execute a Linux command as if we were typing it on the command line.

In [ ]:
! head -5 $NLDN_csv

We see that the file has several columns, each separated by one or more blank spaces (often termed *whitespace*). Since Pandas `read_csv` function's defaults to expecting commas (`,`) as the column separators, we will need to explicitly pass in "one or more blank spaces" as the value of the `sep` argument. Although not intuitive, that value is `'\\s+'`.

There is no header row at the beginning of the file, so we will also need to define a list of abbreviated column `names`; one for each column.

In [ ]:
colNames = ['Date', 'Time', 'Lat', 'Lon', 'Cur', 'Mul', 'CG', 'Chi', 'Maj', 'Axis', 'Min']

#### Use a Jupyterlab [*cell magic* directive](https://ipython.readthedocs.io/en/9.2.0/interactive/magics.html) to determine how long it takes the cell to execute. As we will see, the file has many millions of rows, so it will take a little time!

In [ ]:
%%time
# Read in the CSV file
df = pd.read_csv(f'/spare11/atm533/data/NLDN_{month}.txt',sep='\\s+', names = colNames)

In [ ]:
df

#### Another library gaining traction in the Pangeo ecosystem is [Polars](https://pola.rs). Examine how its `read_csv` method compares to and contrasts with Pandas.

In [ ]:
pl.read_csv?

Let's try reading in the file with Polars. ChatGPT recommended that we state the "one or more blank spaces" as `r"\s+", so let's see if it succeeds. **SPOILER ALERT: It will fail!**

In [ ]:
# Read in the CSV file
df = pl.read_csv(f'/nldn11/combined/archive/NLDN_{month}.txt' ,separator=r"\s+", new_columns = colNames, has_header=False)

#### Why did it fail, and will it ever be an option? Check out this [GitHub Issue thread](https://github.com/pola-rs/polars/issues/12829) with a response from one of the main developers of Polars.

## For large tabular datasets, the convenience of having them represented in a human-readable format such as `csv` is outweighed by larger file sizes, slower read performance, greater system memory usage, and (as we have just seen) lack of support by Polars when columns are separated in non-standard ways. A binary format that solves these issues is called [Parquet](https://parquet.apache.org). 

Convert to Parquet format. Set the output directory and file name.

In [ ]:
outputDir = '.' # Use the current directory
parqFile = f'{outputDir}/NLDN_{month}_full.parquet'

Use one of Pandas file output functions (in this case, [to_parquet](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_parquet.html)) to perform the conversion and output to disk.

In [ ]:
%%time
df.to_parquet(parqFile)

Read in the Parquet file to verify all looks good. First, we'll read it in with Pandas. We'll redefine the dataframe object so we avoid multiple instances of fairly large datasets remaining in system memory.

In [ ]:
%%time 

df = pd.read_parquet(parqFile)

In [ ]:
%%time 

df_polars = pl.read_parquet(parqFile)

How did the load times compare? Try re-running the cells, and also try re-running in different order. Does the comparison change as a result?

Examine the representations of these two dataframes. What do you notice that's similar and different between them?

In [ ]:
df

In [ ]:
df_polars

Next, let's recast and combine some of the columns; particularly, those related to date and time.

In [ ]:
%%time

# Combine the date and time columns into a single series, and then drop the original two series.

dateTimeObj = df['Date'] + ' ' + df['Time']

# Create a new column, now a complete DateTime series
df['DateTime'] = pd.to_datetime(dateTimeObj,format="%Y-%m-%d %H:%M:%S.%f")

# Drop the original two series, as well as some others we are not interested in. This also reduces memory usage!

df = df.drop(columns=['Date','Time','Mul','Chi','Maj','Axis','Min'])

# Reorder the columns

cols = df.columns.tolist()
cols = cols[-1:] + cols[:-1]
df = df[cols]

# Convert the `CG` column into booleans, following https://statisticsglobe.com/convert-string-boolean-pandas-dataframe-column-python .

CGstr = df['CG'].str.strip() # Strip off any leading / trailing whitespace
df['CG'] = CGstr.map({'G': True, 'C': False})

# View the modified dataframe
df

In [ ]:
date_time = df['DateTime']

Let's specify a time range over which we wish to sample lightning events.

In [ ]:
sd =  datetime(2025, 8, 15, 21)
ed =  datetime(2025, 8, 15, 22)

In [ ]:
time_range= pd.date_range(sd, ed)

From the dataframe, select only those times that are within that time range. We'll use Pandas [query](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.query.html) method, which uses database-like syntax to quickly perform subsetting. We'll further only include cloud-to-ground strikes (not intra/inter-cloud flashes).

In [ ]:
%%time
df = df.query('@date_time >= @sd & @date_time <= @ed')
CG = df['CG']
df_subset = df.query('@CG == True')

In [ ]:
df_subset

Now, *georeference* this dataframe, using [Geopandas](https://geopandas.org).

In [ ]:
%%time
lons, lats = df_subset.Lon, df_subset.Lat

gdf = gpd.GeoDataFrame(df_subset,geometry=gpd.points_from_xy(lons,lats))

### In order for data to properly render on an interactive map, we must assign a coordinate reference system to it. Since the coordinates represent latitude and longitude in degrees, we'll use [WGS84 lat-lon projection](https://geopandas.org/en/stable/docs/user_guide/projections.html), aka *EPSG:4326*. 

In [ ]:
gdf.set_crs(epsg=4326, inplace=True, allow_override=True)

# Drop the Lat and Lon columns since they are now handled by the geometry column
gdf = gdf.drop(columns=['Lat', 'Lon'])

gdf

Note the transformed dataframe. What do you think the `geometry` column represents?

## Note! It takes quite a while to render a large number of points on this interactive map. If your subsetted dataframe is more than 100,000 rows, you will definitely want to further restrict your time range, or you could also consider subsetting your geographical extent.

Finally, let's interactively visualize the lightning strikes! You can zoom in/out, pan around, and mouse over individual strikes!

In [ ]:
gdf.explore()

## This is a fairly memory-intensive notebook. Please make sure you *close and shutdown* when done running it!